In [ ]:
import numpy as np
import pandas as pd
import itertools
import h5py
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special
from scipy import integrate
from scipy import interpolate
from scipy import linalg
from scipy import signal
from scipy.optimize import curve_fit
import time
from pathlib import Path
import os
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['text.usetex'] = True

import sys
import mpld3

mpld3.enable_notebook()

sys.path.insert(0, '..')
from modules import structure_func as sf
from modules import regression as rg
from modules import prh

file_path = Path('../data/cosmograil_data/HE0435_Bonvin2016.rdb_.txt')

prh_gen = prh.LightCurvePRHGenerator()
prh_gen.loadQSODataFromFile(file_path)

In [ ]:
plt.errorbar(prh_gen.t, prh_gen.qso_lc_vals, prh_gen.qso_lc_errs, marker='o')

In [ ]:
prh_gen.estimateStructureFunction()
prh_gen.fitSFModel()

In [ ]:
prh_gen.generatePRHDatasetForDelay(delay=50, N_MC_samples=300, outfile=f'../aux/{prh_gen.qso_id}_delay50days.h5')

In [ ]:


qso_id = file_path.name.split('_')[0]
data = pd.read_table(file_path)

t = data['mhjd'].to_numpy(dtype=np.float64)
A = data['mag_A'].to_numpy(dtype=np.float64)
errA = data['magerr_A'].to_numpy(dtype=np.float64)

N = len(t)

In [ ]:
smoothing_val = 1e-1

qso_spline = interpolate.UnivariateSpline(t, A, s=smoothing_val, k=3)
abs_diff = np.abs((qso_spline(t)-A)/A.mean())
frac_err_max = abs_diff.max()
print(frac_err_max)

new_t = np.arange(t[0], t[-1], step=0.2)

print(integrate.simps(qso_spline(new_t)**2, new_t)/(t[-1]-t[0]))

In [ ]:
plt.scatter(t, A)
plt.plot(new_t, qso_spline(new_t), color='red')

In [ ]:
s_vals = [5e-2, 1e-1, 3e-1, 5e-1, 1]
new_t = np.linspace(t[0], t[-1], num=2000)
splines_dict = {}
for s in s_vals:
    spline = interpolate.UnivariateSpline(t, A, s=s, k=3)
    splines_dict[s] = spline

In [ ]:
for s in splines_dict.keys():
    diff = np.abs((splines_dict[s](t) - A)/A.mean())
    print(f's: {s:.2f} FracErrMax = {diff.max()}')

In [ ]:
s = 0.3
plt.plot(new_t, splines_dict[s](new_t), label=f's = {s}')
plt.legend()
plt.scatter(t, A, color='black')

In [ ]:
plt.scatter(t, A, color='b')
plt.plot(new_t, spline(new_t), color='r')

In [ ]:
tau, v = sf.estimate_structure_func_from_data(t, A, errA)

In [ ]:
def power_law_sf(tau, slope, intercept):
    return 10**intercept * tau**slope

def exp_sf(tau, V0, dt0):
    return V0*(1-np.exp(-tau/dt0))

def fit_sf(tau, v, cut_off):
    pars = stats.linregress(np.log10(tau[:cut_off]), np.log10(v[:cut_off]))
    slope = pars.slope
    intercept = pars.intercept
    return slope, intercept

def spline_sf(tau, v):
    spline = interpolate.UnivariateSpline(tau, v, s=1e-6, k=3)
    return spline

In [ ]:
cut_off = int(0.55*len(tau))
slope, intercept = fit_sf(tau, v, cut_off)
popt, pcov = curve_fit(exp_sf, tau[:cut_off], v[:cut_off])

plt.loglog(tau, v, linestyle='None', marker='o', color='g', label='SF')
plt.loglog(tau[:cut_off], v[:cut_off], linestyle='None', marker='o', color='b', label='SF with cutoff')
plt.loglog(tau[:cut_off], power_law_sf(tau[:cut_off], slope, intercept), color='r', 
           label=f'power law: slope {slope:.3f} intercept {intercept:.3f}')
plt.loglog(tau, exp_sf(tau, *popt), color='orange', 
           label=f'exponential: V0 {popt[0]:.3f} dt0 {popt[1]:.3f}')
# v_spline = spline_sf(tau, v)
# plt.loglog(tau, v_spline(tau))
plt.legend(fontsize=12)
print('Max lag considered: %s days ' % tau[cut_off-1])
print('Max existing lag: %s days ' % tau.max())

In [ ]:
# delay = 20
s2 = integrate.simps(spline(new_t)**2, new_t) / (new_t[-1] - new_t[0])
errA_doubled = np.concatenate([errA, errA])
# s2 = (A**2).mean()
curves = []
N = len(t)
counter = 0
for delay in np.arange(50, 60, 1):
    t_doubled = np.concatenate([t, t - delay])
    tau_doubled = sf.compute_lags_matrix(t_doubled)
    C = s2 - power_law_sf(tau_doubled, slope, intercept)
    C += 1e-10*np.eye(2*N)
    eigvals = np.linalg.eigvalsh(C)
    if np.all(eigvals > 0):
        counter += 1
        try:
            L = np.linalg.cholesky(C)
            y = L @ np.random.normal(0, 1, size=2*N) + errA_doubled * np.random.normal(0, 1, size=2*N)
            curves.append([y[:N], y[N:]])
        except np.linalg.LinAlgError:
            print(delay)
            print(eigvals.min())

In [ ]:
idx=9
plt.scatter(t, curves[idx][0] - curves[idx][0].mean(), label='realization %s' % idx, color='r')
plt.scatter(t, A-A.mean(), color='black')

In [ ]:
sim_sfs = []

for idx in range(10):
    _, v_sim = sf.estimate_structure_func_from_data(t, curves[idx][0], errA)
    sim_sfs.append(v_sim)

In [ ]:
plt.loglog(tau, sim_sfs[idx], linestyle='None', marker='o', color='r')
plt.loglog(tau, v, linestyle='None', marker='o', color='black')

In [ ]:
delays = np.arange(1, 100, 1)

curves = []

for delay in delays:
    N = len(t)
    t_doubled = np.concatenate([t, t - delay])
    tau_doubled = sf.compute_lags_matrix(t_doubled)
    A2_mean = (A**2).mean()
    C = A2_mean - exp_sf(tau_doubled, *popt)
    # C = A2_mean - power_law_sf(tau_doubled, slope, intercept)
    # C += 1e-10 * np.eye(2*N)
    errA_doubled = np.concatenate([errA, errA])
    L = np.linalg.cholesky(C)
    y = L @ np.random.normal(0, 1, size=2*N) + errA_doubled**2 * np.random.normal(0, 1, size=2*N)
    curves.append([y[:N], y[N:]])
"""
L = np.linalg.cholesky(C)
y = L @ np.random.normal(0, 1, size=2*N) + errA_doubled**2 * np.random.normal(0, 1, size=2*N)
"""

In [ ]:
eigvals = np.linalg.eigvalsh(C+1e-12*np.eye(2*N))

In [ ]:
plt.imshow(C, cmap='bwr')

In [ ]:
idx = 98
yA = curves[idx][0]
yB = curves[idx][1]

plt.scatter(t, yA-yA.mean(), label='A')
plt.scatter(t, yB-yB.mean(), label='B')
plt.scatter(t, A-A.mean(), label=f'{qso_id} image A')
plt.legend()

In [ ]:
spans = [curves[i][0].max() - curves[i][0].min() for i in range(len(curves))]

In [ ]:
plt.plot(spans)
plt.axhline(y=A.max() - A.min(), color='r')

In [ ]:
N_MC = 300
delta_max = 30
delta_ensemble = np.random.random(size=N_MC) * delta_max

hf = h5py.File(f'{qso_id}_random_delays_0_{delta_max}.h5', 'w')

hf.create_dataset('t_domain', data=t, compression='gzip', compression_opts=9)

t0 = time.time()

for i, delay in enumerate(delta_ensemble):
    N = len(t)
    t_doubled = np.concatenate([t, t - delay])
    tau_doubled = sf.compute_lags_matrix(t_doubled)
    A2_mean = (A**2).mean()
    C = A2_mean - power_law_sf(tau_doubled, slope, intercept)
    errA_doubled = np.concatenate([errA, errA])
    L = np.linalg.cholesky(C)
    y = L @ np.random.normal(0, 1, size=2*N) + errA_doubled**2 * np.random.normal(0, 1, size=2*N)
    
    yA = y[:N]
    yA_delayed = y[N:]
    
    yA = (yA-yA.mean())/yA.std()
    yA_delayed = (yA_delayed-yA_delayed.mean())/yA_delayed.std()
    
    group = hf.create_group(f'realization_{i+1}')
    
    group.create_dataset('delay', data=np.array(delay))
    group.create_dataset('y', data=yA, compression='gzip', compression_opts=9)
    group.create_dataset('y_delayed', data=yA_delayed, compression='gzip', compression_opts=9)
    
tf = time.time()
hf.close()
print(f'Elapsed time: {tf-t0} s')

In [ ]:
def generate_PRH_dataset(t, y, err_y, slope, intercept, delay):
    N = len(t)
    t_doubled = np.concatenate([t, t - delay])
    tau_doubled = sf.compute_lags_matrix(t_doubled)
    y2_mean = (y**2).mean()
    C = y2_mean - power_law_sf(tau_doubled, slope, intercept)
    err_y_doubled = np.concatenate([err_y, err_y])
    L = np.linalg.cholesky(C)
    y = L @ np.random.normal(0, 1, size=2*N) + err_y_doubled**2 * np.random.normal(0, 1, size=2*N)
    
    return y[:N], y[N:]

In [ ]:
hf = h5py.File('RXJ1131_random_delays_0_30.h5', 'r')

In [ ]:
t_dom = hf['t_domain'][()]

In [ ]:
dataset_1 = hf['realization_1']
dataset_5 = hf['realization_5']
dataset_30 = hf['realization_30']

In [ ]:
scaled_data = (A-A.mean())/A.std()

In [ ]:
plt.scatter(t_dom, dataset_1['y'], label='realization 1', color='b')
plt.scatter(t_dom, scaled_data, label=f'true data {qso_id} image A', color='black')
plt.legend()
plt.savefig('realiz1_comparison.pdf')

In [ ]:
plt.scatter(t_dom, dataset_5['y'], label='realization 5', color='r')
plt.scatter(t_dom, scaled_data, label=f'true data {qso_id} image A', color='black')
plt.legend()
plt.savefig('realiz5_comparison.pdf')

In [ ]:
plt.scatter(t_dom, dataset_30['y'], label='realization 30', color='g')
plt.scatter(t_dom, scaled_data, label=f'true data {qso_id} image A', color='black')
plt.legend()
plt.savefig('realiz30_comparison.pdf')

In [ ]:
plt.scatter(t_dom, dataset_5['y'][()], color='b', label='MonteCarlo curve 1')
plt.scatter(t_dom, dataset_5['y_delayed'][()], color='r', 
            label=f"MonteCarlo curve 2: delayed by {dataset_5['delay'][()]:.2f} days")
plt.legend()
plt.savefig('Montecarlo_y_vs_y_delayed.pdf')

In [ ]:
L = np.linalg.cholesky(C)
plt.imshow(np.log10(L), cmap='bwr')

y_f = L @ (np.random.normal(0, 1, size=len(eigvals)))

y_1 = y_f[:N]
y_2 = y_f[N:]

In [ ]:
plt.scatter(t, y_1 - y_1.mean(), label='Montecarlo A')
plt.scatter(t, y_2 - y_2.mean(), label='Montecarlo B')
plt.scatter(t, A - A.mean(), label='true data')
plt.legend()

In [ ]:
tau_sim, v_sim = sf.estimate_structure_func_from_data(t, y_1, errA)

In [ ]:
plt.loglog(tau, v, label='data')
plt.loglog(tau_sim, v_sim, label='Montecarlo')
plt.loglog(tau_sim, 10**intercept * tau_sim**slope)
plt.legend()

In [ ]:
np.random.seed(1234)

R = np.linalg.cholesky(C)

y_new = R @ np.random.normal(0, 1 ,size=2*N)

yA = y_new[:N]
yB = y_new[N:]

yA -= yA.mean()
yB -= yB.mean()

plt.scatter(t, yA, label='Montecarlo A')
plt.scatter(t, yB, label='Montecarlo A shifted')
plt.scatter(t, A-A.mean(), label='true A')
plt.legend()

In [ ]:
kernel = ConstantKernel(2, (1e-3, 1e2)) * Matern(length_scale=200.0, length_scale_bounds=(1, 300), nu=1.5)

gp1 = rg.fit_GP_to_lightcurve(t, yA, errA, kernel)
gp2 = rg.fit_GP_to_lightcurve(t, yB, errA, kernel)

In [ ]:
gp_step = 2
support = np.arange(t[0] - 5e1, t[-1] + 5e1, gp_step)
gpA = rg.fit_GP_to_lightcurve(t, A, errA, kernel)
ypred1, cov1 = gpA.predict(np.expand_dims(support, 1), return_cov=True)

In [ ]:
plt.imshow(cov1, cmap='bwr')

In [ ]:
R = np.linalg.cholesky(cov1)

ynew = R @ np.random.normal(0, 1, size=R.shape[0])

In [ ]:
eigvals, eigvecs = np.linalg.eigh(cov1)
Z = eigvecs

In [ ]:
ynew = Z @ (np.sqrt(eigvals) * np.random.normal(size=len(eigvals)))

In [ ]:
plt.plot(ynew)

In [ ]:
gp_step = 0.2
support = np.arange(t[0] - 5e1, t[-1] + 5e1, gp_step)
y_pred1, sigma1 = gp1.predict(np.expand_dims(support,1), return_std=True)
y_pred2, sigma2 = gp2.predict(np.expand_dims(support,1), return_std=True)

In [ ]:
plt.figure()
plt.plot(t, yB, 'g.', markersize=3, label='MontecarloA')
plt.plot(t, yA, 'b.', markersize=3, label='MontecarloB')
plt.plot(support, y_pred1, 'b-', label='PredictionA')
plt.plot(support, y_pred2, 'g-', label='PredictionB')
plt.fill_between(support, y_pred1 - special.erfinv(0.95)*sigma1, y_pred1 + special.erfinv(0.95)*sigma1,
         alpha=.5, fc='b', ec='None', label='95% confidence interval B')
plt.fill_between(support, y_pred2 - special.erfinv(0.95)*sigma2, y_pred2 + special.erfinv(0.95)*sigma2,
         alpha=.5, fc='g', ec='None', label='95% confidence interval A')
plt.xlabel('$t$')
plt.ylabel('$f(t)$')
plt.legend()

In [ ]:
delay = rg.time_delay_grid_search(y_pred1, y_pred2, sigma1, sigma2, gp_step,
                                  dt_min=0, dt_max=100)
print('Estimated time delay: %s days' % delay)

In [ ]:
t0 = time.time()

true_delays = np.arange(20, 30, 1)
np.random.seed(1234)
simulated_curves = {}
estimated_delays = []
for delay in true_delays:
    print(delay)
    t_doubled = np.concatenate([t, t-delay])
    tau_doubled = sf.compute_lags_matrix(t_doubled)

    y2_mean = (y**2).mean()
    C = y2_mean - power_law_sf(tau_doubled, slope, intercept)
    C += 1e-10*np.eye(C.shape[0])
    R = np.linalg.cholesky(C)
    
    y_new = R @ np.random.normal(0, 1 ,size=2*N)
    yA = y_new[:N]
    yB = y_new[N:]
    yA -= yA.mean()
    yB -= yB.mean()
    """
    
    gp1 = rg.fit_GP_to_lightcurve(t, yA, err_y, kernel)
    gp2 = rg.fit_GP_to_lightcurve(t, yB, err_y, kernel)
    gp_step = 0.2
    support = np.arange(t[0] - 5e1, t[-1] + 5e1, gp_step)
    y_pred1, sigma1 = gp1.predict(np.expand_dims(support,1), return_std=True)
    y_pred2, sigma2 = gp2.predict(np.expand_dims(support,1), return_std=True)
    estimated_delay = rg.time_delay_grid_search(y_pred1, y_pred2, sigma1, sigma2, gp_step,
                                                dt_min=0, dt_max=100)
    estimated_delays.append(estimated_delay)
    """
    
    simulated_curves[delay] = [yA, yB]
    

estimated_delays = np.array(estimated_delays)
tf = time.time()
print('Elapsed time: %s' % (tf - t0))

In [ ]:
plt.scatter(true_delays, estimated_delays)

In [ ]:
d = 20
plt.scatter(t, simulated_curves[d][0], label='ref delay %s ' % d)
plt.scatter(t, simulated_curves[d][1], label='shifted')
plt.scatter(t, y-y.mean(), label='HE0435')
plt.legend()

In [ ]:
A = np.array([[3,2],[2,4]])

In [ ]:
eigvals, eigvecs = np.linalg.eigh(A)

In [ ]:
Z = eigvecs.T
sqrtLambda = np.diag(np.sqrt(eigvals))

In [ ]:
np.linalg.qr(sqrtLambda @ Z)

In [ ]:
np.linalg.cholesky(A)

In [ ]:
Z.T @ np.diag(eigvals) @ Z